In [54]:
TRAIN_DATASET_PATH = '/Users/jjtiong/Desktop/captcha_dataset/train'
TEST_DATASET_PATH = '/Users/jjtiong/Desktop/captcha_dataset/test'
OUTPUT_PATH = '/Users/jjtiong/Desktop/captcha_dataset/segmented_train'
SEGMENTED_TRAIN_DATASET_PATH = '/Users/jjtiong/Desktop/captcha_dataset/segmented_train'
AUGMENTED_SEGMENTED_TRAIN_DATASET_PATH = '/Users/jjtiong/Desktop/captcha_dataset/augmented_segmented_train'
MODEL_PATH = '/Users/jjtiong/Desktop/captcha_dataset/model2/character_recognition_model.h5'

In [55]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


def gaussian_thresholding(image):
    # Use gaussian blur to make characters thicker before thresholding 
    blur = cv2.GaussianBlur(image, (5, 5), 0)
    thresholded_image = cv2.adaptiveThreshold(
        blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, blockSize=11, C=2
    )
    return image

def dilate(image):
    #  Remove noise via dilation
    kernel = np.ones((3,3), np.uint8)
    dilation = cv2.dilate(image, kernel, iterations=1)
    
    return dilation

# def adaptive_thresholding(image):
#     thresholded_image = cv2.adaptiveThreshold(
#         image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
#         cv2.THRESH_BINARY, blockSize=11, C=2
#     )
#     return thresholded_image

# def remove_noise(image):
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
#     cleaned_image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations=1)
#     cleaned_image = cv2.morphologyEx(cleaned_image, cv2.MORPH_CLOSE, kernel, iterations=1)
#     return cleaned_image

# def remove_lines(image):
#     lines = cv2.HoughLinesP(image, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)
#     if lines is not None:
#         for line in lines:
#             x1, y1, x2, y2 = line[0]
#             cv2.line(image, (x1, y1), (x2, y2), 0, thickness=2)
#     return image

def segment_characters(image):
    inverted_image = cv2.bitwise_not(image)
    contours, _ = cv2.findContours(inverted_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    char_images = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if h > 10:  # Filter small noise
            char_img = image[y:y+h, x:x+w]
            char_images.append((x, char_img))
    char_images = sorted(char_images, key=lambda c: c[0])  # Sort by x-coordinate

    return [char[1] for char in char_images]

def process_and_save_images(input_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Keep track of how many captchas were skipped
    skipped_count = 0

    # Process each image in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):
            # Extract the true label from the filename
            base_name = filename.split('-')[0]

            # Load and preprocess the image
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Apply preprocessing steps
            image = gaussian_thresholding(image)
            image = dilate(image)
            # image = remove_lines(image)

            # Segment characters
            char_images = segment_characters(image)

            # Validate the number of segmented parts
            if len(char_images) != len(base_name):
                print(f"Skipping {filename}: Expected {len(base_name)} parts, got {len(char_images)}.")
                skipped_count += 1
                continue  # Skip saving this image

            # Save each segmented character follow the naming convention <label_str>-<char>.png
            for idx, char_img in enumerate(char_images):
                char_label = base_name[idx]
                save_name = f"{base_name}-{char_label}.png"
                save_path = os.path.join(output_dir, save_name)

                # Save the segmented character
                cv2.imwrite(save_path, char_img)

    print(f"Processing complete! Segmented characters are saved in {output_dir}")
    print(f"Total skipped: {skipped_count}")


process_and_save_images(TRAIN_DATASET_PATH, SEGMENTED_TRAIN_DATASET_PATH)


Skipping 2zimhl0-0.png: Expected 7 parts, got 0.
Skipping stx1upye-0.png: Expected 8 parts, got 0.
Skipping nip6-0.png: Expected 4 parts, got 1.
Skipping vmii-0.png: Expected 4 parts, got 5.
Skipping 4rcb-0.png: Expected 4 parts, got 0.
Skipping yyvv0kit-0.png: Expected 8 parts, got 1.
Skipping y0cq-0.png: Expected 4 parts, got 2.
Skipping hdu5-0.png: Expected 4 parts, got 1.
Skipping 8d2y98-0.png: Expected 6 parts, got 5.
Skipping aswhu3-0.png: Expected 6 parts, got 7.
Skipping b764czl-0.png: Expected 7 parts, got 4.
Skipping qkyk15f-0.png: Expected 7 parts, got 6.
Skipping dwzv-0.png: Expected 4 parts, got 0.
Skipping 9a2xdo-0.png: Expected 6 parts, got 0.
Skipping usskt-0.png: Expected 5 parts, got 0.
Skipping pk0k8c3-0.png: Expected 7 parts, got 10.
Skipping 0hhit-0.png: Expected 5 parts, got 7.
Skipping g0o1-0.png: Expected 4 parts, got 1.
Skipping ts90mw-0.png: Expected 6 parts, got 11.
Skipping wg3pj3rn-0.png: Expected 8 parts, got 7.
Skipping i5t4f5-0.png: Expected 6 parts, got

In [56]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_and_save_images(input_dir, output_dir, augment_count=3):
    """Augment images from `input_dir` and save them with proper naming to `output_dir`."""
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    # Define a more controlled data augmentation pipeline
    datagen = ImageDataGenerator(
        rotation_range=10,          # Small rotation for better quality
        width_shift_range=0.05,     # Slight horizontal shift
        height_shift_range=0.05,    # Slight vertical shift
        shear_range=0.1,            # Mild shearing
        zoom_range=0.1,             # Mild zoom
        fill_mode='nearest'         # Avoid introducing artifacts
    )

    # Loop through all images in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):
            # Load the original image
            image_path = os.path.join(input_dir, filename)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Expand dimensions for augmentation
            img = np.expand_dims(img, axis=-1)  # Add channel dimension
            img = np.expand_dims(img, axis=0)   # Add batch dimension

            # Extract the base filename without extension (e.g., 'abc-o')
            base_name = filename.split('.')[0]

            # Generate and save augmented images
            for i, batch in enumerate(datagen.flow(img, batch_size=1)):
                # Save each augmented image with proper naming
                augmented_filename = f"{base_name}-{i + 1}.png"
                augmented_path = os.path.join(output_dir, augmented_filename)

                # Save the augmented image
                cv2.imwrite(augmented_path, batch[0].squeeze().astype(np.uint8))

                # Stop after generating `augment_count` images per original image
                if i + 1 >= augment_count:
                    break

    print(f"Augmentation complete! Augmented images saved in {output_dir}")


# Example usage
input_dir = SEGMENTED_TRAIN_DATASET_PATH
output_dir = AUGMENTED_SEGMENTED_TRAIN_DATASET_PATH
augment_and_save_images(input_dir, output_dir)


Augmentation complete! Augmented images saved in /Users/jjtiong/Desktop/captcha_dataset/augmented_segmented_train


In [57]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# Define character set (e.g., 0-9, a-z)
CHAR_SET = "0123456789abcdefghijklmnopqrstuvwxyz"
CHAR_TO_IDX = {char: idx for idx, char in enumerate(CHAR_SET)}

def load_dataset(data_dir):
    images = []
    labels = []

    for filename in os.listdir(data_dir):
        if filename.endswith('.png'):
            # Extract the character from the filename (e.g., 'o' from 'oq1j-o.png')
            char_label = filename.split('-')[1][0]

            if char_label in CHAR_TO_IDX:
                # Load the image in grayscale
                image_path = os.path.join(data_dir, filename)
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

                # Resize to 28x28 (if needed)
                img = cv2.resize(img, (28, 28))
                img = img / 255.0  # Normalize pixel values to [0, 1]

                # Store the image and label
                images.append(img)
                labels.append(CHAR_TO_IDX[char_label])

    # Convert to numpy arrays
    images = np.array(images).reshape(-1, 28, 28, 1)  # Add channel dimension
    labels = np.array(labels)

    return images, labels

# Load the dataset
images, labels = load_dataset(output_dir)
print(f"Loaded {len(images)} images with {len(set(labels))} unique labels.")

Loaded 88374 images with 36 unique labels.


In [58]:
# Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}, Validation set size: {X_val.shape[0]}")


Training set size: 70699, Validation set size: 17675


In [59]:
#MODEL 1 ~8%

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# def build_model(num_classes):
#     model = Sequential([
#         Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
#         MaxPooling2D((2, 2)),
#         Dropout(0.2),

#         Conv2D(64, (3, 3), activation='relu'),
#         MaxPooling2D((2, 2)),
#         Dropout(0.2),

#         Flatten(),
#         Dense(128, activation='relu'),
#         Dropout(0.5),
#         Dense(num_classes, activation='softmax')
#     ])

#     model.compile(optimizer='adam',
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])
#     return model

# # Build the model for 36 classes (0-9, a-z)
# num_classes = len(CHAR_SET)
# model = build_model(num_classes)
# model.summary()

In [60]:
#MODEL 2 ~24%

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, ReLU

# def build_custom_cnn(num_classes):
#     model = Sequential()

#     # First Conv Block
#     model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     # Second Conv Block
#     model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     # Third Conv Block
#     model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     # Fully Connected Layers
#     model.add(Flatten())
#     model.add(Dropout(0.5))
#     model.add(Dense(128, activation='relu'))
#     model.add(Dense(36, activation='softmax'))  # 36 classes: 0-9 + a-z

#     # Compile the model
#     model.compile(optimizer='adam', 
#                   loss='sparse_categorical_crossentropy', 
#                   metrics=['accuracy'])

#     return model


# num_classes = len(CHAR_SET)
# model = build_model(num_classes)
# model.summary()


In [61]:
#MODEL 3 ~33% (AlexNet)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU

def build_alexnet():
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(64, (3, 3), strides=1, padding='same', input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 2nd Convolutional Layer
    model.add(Conv2D(128, (3, 3), strides=1, padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 3rd Convolutional Layer
    model.add(Conv2D(256, (3, 3), strides=1, padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the output for Fully Connected Layers
    model.add(Flatten())

    # 1st Fully Connected Layer
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    # 2nd Fully Connected Layer
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(36, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

model = build_alexnet()
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_6 (ReLU)                  │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_7 (ReLU)                  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 7, 7, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_8 (ReLU)                  │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 36)             │        18,468 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,832,740 (6.99 MB)

 Trainable params: 1,831,844 (6.99 MB)

 Non-trainable params: 896 (3.50 KB)

In [62]:
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=30,
#     batch_size=64
# )
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model with EarlyStopping
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=[early_stopping]
)

Epoch 1/30
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 65s 58ms/step - accuracy: 0.1909 - loss: 2.9335 - val_accuracy: 0.6095 - val_loss: 1.4479
Epoch 2/30
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 63s 57ms/step - accuracy: 0.5775 - loss: 1.3941 - val_accuracy: 0.5969 - val_loss: 1.4257
Epoch 3/30
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 64s 58ms/step - accuracy: 0.6947 - loss: 1.0297 - val_accuracy: 0.7207 - val_loss: 0.9415
Epoch 4/30
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 63s 57ms/step - accuracy: 0.7390 - loss: 0.8602 - val_accuracy: 0.7407 - val_loss: 0.8857
Epoch 5/30
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 63s 57ms/step - accuracy: 0.7696 - loss: 0.7649 - val_accuracy: 0.7290 - val_loss: 0.9569
Epoch 6/30
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 65s 58ms/step - accuracy: 0.7911 - loss: 0.6888 - val_accuracy: 0.8074 - val_loss: 0.6516
Epoch 7/30
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 66s 60ms/step - accuracy: 0.8063 - loss: 0.6269 - val_accuracy: 0.7285 - val_loss: 0.9352
Epoch 8/30
1105/1105 ━━━━━━━━━━━━━━━━━━━━ 68s 62ms/step - accuracy: 0.8169 -

In [63]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%")


553/553 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.8258 - loss: 0.5970
Validation Loss: 0.5959, Validation Accuracy: 82.66%


In [64]:
# Save the trained model
def save_model(model, dir_path, model_name="character_recognition_model.h5"):
    # Ensure the directory exists
    os.makedirs(dir_path, exist_ok=True)
    
    # Create the full path to the model file
    model_path = os.path.join(dir_path, model_name)
    
    # Save the model
    model.save(model_path)
    print(f"Model saved successfully at: {model_path}")

# Example usage
save_dir = '/Users/jjtiong/Desktop/captcha_dataset/model2'
save_model(model, save_dir)

Model saved successfully at: /Users/jjtiong/Desktop/captcha_dataset/model2/character_recognition_model.h5


In [65]:
def predict_character(image_path, model):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28)) / 255.0  # Normalize
    img = np.expand_dims(img, axis=[0, -1])  # Add batch and channel dimensions

    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    predicted_char = CHAR_SET[predicted_label]

    return predicted_char

# Example usage
model = tf.keras.models.load_model(MODEL_PATH)
predicted_char = predict_character('/Users/jjtiong/Desktop/captcha_dataset/segmented_train/0tqk-t.png', model)
print(f"Predicted Character: {predicted_char}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Predicted Character: t


In [66]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model(MODEL_PATH)

# Define the character set used during training
CHAR_SET = "0123456789abcdefghijklmnopqrstuvwxyz"
CHAR_MAP = {idx: char for idx, char in enumerate(CHAR_SET)}

def preprocess_image(image_path):
    """Preprocess the image using the same steps as in training."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = gaussian_thresholding(image)
    image = dilate(image)
    return image

def gaussian_thresholding(image):
    # Use gaussian blur to make characters thicker before thresholding 
    blur = cv2.GaussianBlur(image, (5, 5), 0)
    thresholded_image = cv2.adaptiveThreshold(
        blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, blockSize=11, C=2
    )
    return image

def dilate(image):
    #  Remove noise via dilation
    kernel = np.ones((3,3), np.uint8)
    dilation = cv2.dilate(image, kernel, iterations=1)
    
    # show_image("Dilated", dilation)
    return dilation

# def adaptive_thresholding(image):
#     """Apply adaptive thresholding."""
#     return cv2.adaptiveThreshold(
#         image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
#         cv2.THRESH_BINARY_INV, blockSize=11, C=2
#     )

# def remove_noise(image):
#     """Remove small noise using morphological operations."""
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
#     cleaned_image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations=1)
#     cleaned_image = cv2.morphologyEx(cleaned_image, cv2.MORPH_CLOSE, kernel, iterations=1)
#     return cleaned_image

# def remove_lines(image):
#     """Remove lines using Hough Line Transform."""
#     lines = cv2.HoughLinesP(image, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)
#     if lines is not None:
#         for line in lines:
#             x1, y1, x2, y2 = line[0]
#             cv2.line(image, (x1, y1), (x2, y2), 0, thickness=2)
#     return image

def segment_characters(image):
    """Segment the CAPTCHA image into individual characters."""
    inverted_image = cv2.bitwise_not(image)

    contours, _ = cv2.findContours(inverted_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    char_images = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if h > 10:  # Ignore small contours (noise)
            char_img = image[y:y+h, x:x+w]
            char_images.append((x, char_img))

    # Sort by x-coordinate to ensure correct order
    char_images = sorted(char_images, key=lambda c: c[0])
    return [char[1] for char in char_images]

def predict_character(image, model):
    """Predict a single character using the trained model."""
    img = cv2.resize(image, (28, 28)) / 255.0  # Resize and normalize
    img = np.expand_dims(img, axis=[0, -1])  # Add batch and channel dimensions

    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    predicted_char = CHAR_MAP[predicted_label]
    return predicted_char

def predict_captcha_string(image_path, model):
    """Predict the entire CAPTCHA string from the input image."""
    # Preprocess the image
    processed_image = preprocess_image(image_path)

    # Segment the characters
    char_images = segment_characters(processed_image)

    # Predict each character and reconstruct the string
    predicted_string = ""
    for char_img in char_images:
        predicted_char = predict_character(char_img, model)
        predicted_string += predicted_char

    return predicted_string



# Example usage
captcha_image_path = "/Users/jjtiong/Desktop/captcha_dataset/test/2d76-0.png"
predicted_string = predict_captcha_string(captcha_image_path, model)
print(f"Predicted CAPTCHA string: {predicted_string}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Predicted CAPTCHA string: 2d76


In [67]:
import os

def evaluate_on_unseen_data(unseen_dir, model, num_images=20):
    """Evaluate the model on the first `num_images` from the unseen directory."""
    correct_predictions = 0
    total_images = 0

    # Get the first `num_images` files from the directory
    all_files = [f for f in os.listdir(unseen_dir) if f.endswith('.png')]
    all_files = sorted(all_files)[:num_images]  # Limit to first `num_images`

    for filename in all_files:
        # Extract the true label from the filename (e.g., '2d76' from '2d76-0.png')
        true_label = filename.split('-')[0]

        # Get the full path to the image
        image_path = os.path.join(unseen_dir, filename)

        # Predict the CAPTCHA string
        predicted_string = predict_captcha_string(image_path, model)
        print(f"True: {true_label}, Predicted: {predicted_string}")

        # Compare the predicted string with the true label
        if predicted_string == true_label:
            correct_predictions += 1

        total_images += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_images * 100
    print(f"\nAccuracy on the first {num_images} images: {accuracy:.2f}%")

# Example usage
unseen_dir = "/Users/jjtiong/Desktop/captcha_dataset/test"
evaluate_on_unseen_data(unseen_dir, model, 1000)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
True: 002e23, Predicted: 002e23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
True: 03yl9s, Predicted: 03yi9s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
True: 03yuav5, Predicted: 03yuav5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━